# Get the current EUR price in PLN using NexosAPI and web search

This notebook shows how to:
- Configure logging and load environment variables for NexosAPI
- Construct a chat completion request that uses a web search tool to fetch the latest exchange rate
- Send the request and log the structured JSON response

Prerequisites:
- A valid NexosAPI configuration and credentials (e.g., stored in a `.env` file)
- Internet access for the web search tool

## Setup: logging and environment variables

Initialize verbose logging to observe SDK behavior and point NexosAPI to your `.env` file so it can load credentials and configuration. Update the `NEXOSAI_INIT__DOTENV_PATH` to match your environment.

In [1]:
import json
import logging
import os

# Set the logging level to desired verbosity
# This will help in debugging by showing detailed logs of the operations
# and interactions with the API.
from nexosapi.common.logging import setup_logging

LOGGING_LEVEL = logging.INFO
setup_logging(level=LOGGING_LEVEL)

# Set the environment variable to use dotenv to load environment variables

os.environ["NEXOSAI_INIT__DOTENV_PATH"] = (
    "/Users/kamilrybacki/Projects/nexos.api/examples/.env"  # Set the path to the .env file
)
os.environ["NEXOSAI_INIT__LOAD_DOTENV"] = "true"

## Build a request that uses the web search tool

Create a chat completion request that:
- Provides a system instruction to search for the current EUR→PLN rate
- Asks the user question
- Enables the web search tool with a medium search context
- Logs the prepared payload for inspection before sending

In [2]:
from nexosapi.api.endpoints import chat

# Start with a new request
tool_request = {
    "model": "7071fb76-722a-41ad-8477-68123b04b4ab",
    "messages": [
        {
            "role": "system",
            "content": """
            You are a helpful assistant that provides current currency prices
            based on the latest available data. You can use a Google search tool
            to find the most recent exchange rates. When asked for a currency price,
            you will search for the current price of the specified currency in the
            specified currency. For example, if asked for the price of an euro in Polish Zlot
            (PLN), you will search for "current price of euro in PLN" and provide the
            latest exchange rate you find.

            If you cannot find the price, respond with an error message:
            "Could not find the current price of the specified currency."
        """,
        },
    ],
}
# Prepare the new request with the specified parameters
chat.completions.request.prepare(tool_request)

# Add a Google search tool to the request
chat.completions.request.with_search_engine_tool(
    options={
        "search_context_size": "medium",
    },
)

# Show the request before sending it
tool_request_to_send = chat.completions.request.dump()
print(json.dumps(tool_request_to_send, indent=2, ensure_ascii=False, sort_keys=True))

2025-08-17 21:12:16,298 - root - INFO - [SDK] Dependencies automatically wired.


{
  "frequency_penalty": 0.0,
  "messages": [
    {
      "content": "\n            You are a helpful assistant that provides current currency prices\n            based on the latest available data. You can use a Google search tool\n            to find the most recent exchange rates. When asked for a currency price,\n            you will search for the current price of the specified currency in the\n            specified currency. For example, if asked for the price of an euro in Polish Zlot\n            (PLN), you will search for \"current price of euro in PLN\" and provide the\n            latest exchange rate you find.\n\n            If you cannot find the price, respond with an error message:\n            \"Could not find the current price of the specified currency.\"\n        ",
      "role": "system"
    }
  ],
  "modalities": [
    "text"
  ],
  "model": "7071fb76-722a-41ad-8477-68123b04b4ab",
  "n": 1,
  "presence_penalty": 0.0,
  "service_tier": "auto",
  "temperature": 1.0,
 

## Send the request and log the response

Reload the last prepared request and send it asynchronously. The response is logged and should include a JSON payload with the currency and its price in PLN (or an error message). Note: Jupyter supports top-level `await`.

In [3]:
# Send the new request to the chat completions endpoint with the Google search tool
chat.completions.request.reload_last()
tool_response = await chat.completions.request.send()

# Get the response from the chat completions endpoint
print(json.dumps(tool_response.model_dump(), indent=2, ensure_ascii=False, sort_keys=True))

2025-08-17 21:12:26,095 - httpx - INFO - HTTP Request: POST https://api.nexos.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-17 21:12:26,177 - root - INFO - [SDK] Thinking process detected for message from None


{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "annotations": [
          {
            "type": "url_citation",
            "url_citation": {
              "title": "Bloomberg | The EUR-PLN X-RATE is listed at 4.2600, updated as of 12:00 AM EDT 08/15/25.",
              "url": "https://www.bloomberg.com/quote/EURPLN:CUR"
            }
          },
          {
            "type": "url_citation",
            "url_citation": {
              "title": "European Central Bank | ECB euro reference exchange rate: 1 EUR = 4.2605 PLN on 14 August 2025.",
              "url": "https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-pln.en.html"
            }
          }
        ],
        "content": "The current exchange rate for 1 euro (EUR) is approximately 4.26 Polish zloty (PLN) as of the latest market data.",
        "role": "assistant",
        "thinking": "<think>\n1. My task is to provi